In [55]:
import pandas as pd
import numpy as np

In [57]:
#pip install xgboost


In [34]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [4]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

In [58]:
Session1_oasis2_atropy_20240418_before_fastsurfer.csv

'/home/limseoyoung/debug/For_tabulardata_testing'

In [78]:

#brain_vars_df = pd.read_csv('MPR3_oasis2_atropy_20240417_before_fastsurfer.csv')
brain_vars_df = pd.read_csv('/home/limseoyoung/debug/Session4_oasis2_atropy_20240418_before_fastsurfer.csv')
phenotype_df = pd.read_csv('oasis_longitudinal_oasis2.csv')



In [79]:
phenotype_df['MRI ID'] = phenotype_df['MRI ID'].str.strip()
phenotype_df = phenotype_df.dropna(subset=['Demention'])  # Drop rows where target value is NaN
phenotype_df['Demention'] = phenotype_df['Demention'].astype(int)


#brain_vars_df['subject_clean'] = brain_vars_df['subject'].str.extract(r'(OAS2_\d+_MR1)')
brain_vars_df['subject_clean'] = brain_vars_df['subject'].str.extract(r'(OAS2_\d+_MR\d)')


merged_data = pd.merge(brain_vars_df, phenotype_df, left_on='subject_clean', right_on='MRI ID')
# 뇌만 가지고 해야하니까 , 다른거 다 빼고할것!!! 
non_feature_columns = ['subject', 'subject_clean', 'Subject ID', 'MRI ID', 'Group', 'Visit', 'MR Delay', 'Hand','Demention', 'Age','EDUC',
 'SES',
 'MMSE',
 'CDR',
 'eTIV',
 'nWBV',
 'ASF', 'unknwon','HippocampusOccupancyScore','HippocampusAsymmetryIndex']
numeric_brain_var_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns.tolist()
numeric_brain_var_columns = [col for col in numeric_brain_var_columns if col not in non_feature_columns]


scaler = StandardScaler()
merged_data[numeric_brain_var_columns] = scaler.fit_transform(merged_data[numeric_brain_var_columns])

In [80]:
X = merged_data[numeric_brain_var_columns]
y = merged_data['Demention']

In [81]:
X = X.drop(columns = ['Unnamed: 0'])

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train the classifier
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_classifier.predict(X_test)

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [83]:

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
# create dataset

cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
# define the model
best_model = XGBClassifier(random_state=1)
# define search space"
space = dict()
space['n_estimators'] = [10, 50,100, 500]
space['max_features'] = ['sqrt', 'log2']
space['max_depth'] = [6, 8, 10, 12,20]
    

# define search
search = GridSearchCV(best_model, space, scoring="precision", n_jobs=1, cv=cv_inner, refit=True)
# configure the cross-validation procedure
cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)
# execute the nested cross-validation
scores = cross_val_score(search, X, y, scoring= "precision", cv=cv_outer, n_jobs=-1)
# report performance
print('Auc: %.3f (%.3f)' % (mean(scores), std(scores)))

/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:44:56] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:44:56] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:44:56] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:44:56] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:44:56] WARNING: /workspace/src/learner.

Auc: 0.700 (0.400)


/home/limseoyoung/fsl/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:44:58] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:44:58] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/home/limseoyoung/fsl/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavio

In [11]:
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [12]:
from sklearn.model_selection import KFold, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
#import shap
import matplotlib.pyplot as plt
from numpy import mean, std

In [13]:
# Define the cross-validation procedures
cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)

# Define the model and search space
model = XGBClassifier(random_state=1)
# define search space"
space = {'n_estimators' : [10, 50,100, 500],
        'max_features':['sqrt', 'log2'],
        'max_depth': [6, 8, 10, 12,20]}


# Initialize a list to store feature importances for each outer fold
feature_importances = []

# Loop over the folds defined by the outer cross-validation
for train_ix, test_ix in cv_outer.split(X):
    # Split data
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]

    # Define search
    search = GridSearchCV(model, space, scoring='roc_auc', n_jobs=1, cv=cv_inner, refit=True)
    
    # Fit the model
    search.fit(X_train, y_train)
    
    # Get the best model
    best_model = search.best_estimator_
    
    # Store feature importances
    feature_importances.append(best_model.feature_importances_)


# Compute average feature importance
average_importance = np.mean(feature_importances, axis=0)

# Sort features by importance
sorted_indices = np.argsort(average_importance)[::-1]

# Determine the number of top features to select (top 10%)
top_features_count = int(len(average_importance) * 0.1)


top_indices = sorted_indices[:top_features_count]

# Plot feature importances of the top 10%
plt.figure(figsize=(12, 6))
plt.title('Top 10% Feature Importances')
plt.bar(range(top_features_count), average_importance[top_indices], align='center')
plt.xticks(range(top_features_count), X.columns[top_indices], rotation=90)
plt.show()

/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:52:09] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:52:18] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:52:29] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:52:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/home/limseoyoung/fsl/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:52:49] WARNING: /workspace/src/learner.

KeyboardInterrupt: 

In [ ]:
#Auc: 0.627 (0.093)-> MPR1 , temporal 3rdventricle

In [ ]:
# Auc: 0.544 (0.100)-> MPR2, 4th ventricle, pallidum 

In [ ]:
#  Auc: 0.634 (0.098) -> MPR 3, CS , insula 

In [86]:
(0.530 + 0.477 + 0.615 ) / 3

0.5406666666666667

In [87]:
( 0.581 + 0.638 +0.549  ) / 3

0.5893333333333333